In [1]:
from datetime import datetime, timedelta
import pandas as pd
import sqlalchemy
import csv
import os

In [2]:
user_name = 'KHU9683'
pword = 'Zen18@dme'
host_name='PARAEDWPROD.DW.MEDCITY.NET'

In [3]:
td_engine = sqlalchemy.create_engine('teradatasql://' + host_name + '/?user=' + user_name + '&password=' + pword + '&logmech=KRB5')

In [4]:
def touch(path):
    with open(path, 'a') as csvfile:
        os.utime(path, None)
        # close()

In [5]:
#Write csvfile to local directory
def write_csv_file(path, fields, data, write_mode="w", delimiter=",", quote_char=""):
    
    with open(path, write_mode, newline='') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=delimiter, quotechar=quote_char, quoting=csv.QUOTE_MINIMAL)
        # csvwriter.writerow(fields)
        csvwriter.writerows(data)

In [6]:
#path of file with table name and database name in csv file 
input_path = "InputFiles\\copy_table_data.csv"
output_path_folder = "OutputFiles\\TableData\\"
output_file_extension = "csv"
v_chunksize = 100000
write_mode = "a"

In [7]:
#get the DDLs for the tables from teradata 
def gen_copy_table_data():
        dt1 = datetime.now()
        delimiter = ','
        quote_char = '"'
        df = pd.read_csv(input_path, index_col=None)
        run_date = (datetime.now()+timedelta(hours=-6)).strftime('%y%m%d')
        if not os.path.exists(output_path_folder + run_date):
            os.makedirs(output_path_folder + run_date)      

        for index, row in df.iterrows():
            try:
                database_name = str(row['Database']).strip().lower()
                table_name = str(row['Table']).strip().lower()

                query = "Select LOWER(TRIM(columnname)) AS columnname, UPPER(TRIM(columntype)) AS columntype "\
                    "From dbc.columnsV WHERE UPPER(DatabaseName) = '{}' AND UPPER(TableName) = '{}'"\
                    ";".format(database_name, table_name)
                tablemeta_df = pd.read_sql(query, td_engine)

                query = "Select * From {}.{};".format(database_name, table_name)

                output_path = "{}{}\\{}_{}.{}".format(output_path_folder, run_date, table_name, run_date, output_file_extension)

                touch(output_path)
                
                for results_df in pd.read_sql(query, td_engine, chunksize=v_chunksize):
                
                    # print(tablemeta_df[tablemeta_df["columnname"] == "sk_code"])

                    for column in results_df:
                        column_name = str(results_df[column].name).lower()
                        column_type = str(tablemeta_df[tablemeta_df['columnname']==column_name]['columntype'].values[0])
                        # print(column_type)

                        if column_type in ['CV', 'CF', 'BO', 'BF', 'BV']:
                            # print(column_name)
                            results_df[column] = results_df[column].str.encode("ascii","ignore").str.decode("ascii")
                            # results_df[column].replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

                    fields = list(results_df.columns)

                    data = results_df.values.tolist()
                    
                    write_csv_file(output_path, fields, data, write_mode, delimiter, quote_char)

            except Exception as e1:
                print(e1)
                print("Database : {}, Table : {}".format(database_name,table_name))
                pass
        
        dt2 = datetime.now()
        print(dt2-dt1)
        print("Tables for Copying Data : ", len(df))

In [8]:
print("Begin of Processing")

gen_copy_table_data()

print("End of Processing")

Begin of Processing
0:15:58.097538
Tables for Copying Data :  22
End of Processing
